In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from keras.models import Sequential 
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

2024-11-07 10:49:59.786149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730998199.799929    4923 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730998199.803979    4923 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 10:49:59.819684: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 1. Importar la base de datos del circuito RC
RC_circuit_db = pd.read_csv('./datasets/db_completa_RC_Tecnologia.csv')
RC_circuit_db.head()

,time,voltaje_ent,voltaje_cap_5,R_5,C_5,voltaje_cap_8,R_8,C_8,voltaje_cap_10,R_10,C_10,voltaje_cap_12,R_12,C_12,voltaje_cap_15,R_15,C_15
0,0,4.93,0.00,55300,0.0001,0.00,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
1,200,4.93,0.00,55300,0.0001,0.12,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
2,400,4.93,0.02,55300,0.0001,0.30,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
3,600,4.93,0.24,55300,0.0001,0.50,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
4,800,4.93,0.43,55300,0.0001,0.66,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.04,14920,0.001


In [ ]:
# 2. Normalizar mis datos

In [3]:
def normalize(x, xmin, xmax):
    return (xmax - xmin)*((x - np.min(x))/(np.max(x) - np.min(x))) + xmin

In [4]:
#X = [tiempo, R, C, Ve]
#y = [Vc]

xmin = 0.1
xmax = 0.8

R = normalize(np.hstack([RC_circuit_db['R_5'], RC_circuit_db['R_8'], RC_circuit_db['R_10'], RC_circuit_db['R_12'], RC_circuit_db['R_15']]), xmin, xmax)
C = normalize(np.hstack([RC_circuit_db['C_5'], RC_circuit_db['C_8'], RC_circuit_db['C_10'], RC_circuit_db['C_12'], RC_circuit_db['C_15']]), xmin, xmax)
t = normalize(np.hstack([RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time']]), xmin, xmax)
Ve = np.hstack([RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent']])

# Input variables
X = np.vstack([R, C, t, Ve]).T

# Target
y = np.hstack([RC_circuit_db['voltaje_cap_5'], RC_circuit_db['voltaje_cap_8'], RC_circuit_db['voltaje_cap_10'], RC_circuit_db['voltaje_cap_12'], RC_circuit_db['voltaje_cap_15']])

In [5]:
# 3. Segmentar la base de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2002)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14000, 4)
(6000, 4)
(14000,)
(6000,)


In [6]:
# 4. Definimos la topologia de la ANN
n_neurons = 2
model = Sequential(name = "Model-with-Four-Inputs") # Model
model.add(Input(shape=(X.shape[1],), name = 'Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Dense(n_neurons, activation='sigmoid', name = 'Hidden-Layer')) # Hidden Layer
model.add(Dense(1, activation = 'linear', name = 'Output-Layer')) # Output Layer}
model.summary()

I0000 00:00:1730998214.785196    4923 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2173 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "Model-with-Four-Inputs"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden-Layer (Dense)            │ (None, 1)              │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output-Layer (Dense)            │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7 (28.00 B)

 Trainable params: 7 (28.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 5. Compile keras model
model.compile(optimizer = 'sgd', # default='rmsprop', an algorithm to be used in backpropagation
              loss = 'mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics = ['mean_squared_error','mean_absolute_error'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
            )

In [8]:
# 6. Entrenamos el modelo
history = model.fit(X_train, # input data
                    y_train, # target data
                    #batch_size = 10, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs = 3500, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose = 'auto', # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    #callbacks = [earlyStopping, mcp_save],
                    validation_data = (X_test, y_test))

Epoch 1/3500
  4/438 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.2460 - mean_absolute_error: 2.2350 - mean_squared_error: 6.24602460

I0000 00:00:1730998294.775769    5000 service.cc:148] XLA service 0x74b3b4005f30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730998294.776022    5000 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-11-07 10:51:34.792949: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730998294.827355    5000 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1730998294.948174    5000 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.2061 - mean_absolute_error: 1.5623 - mean_squared_error: 3.2061 - val_loss: 2.8968 - val_mean_absolute_error: 1.4891 - val_mean_squared_error: 2.8968
Epoch 2/3500
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 2.8724 - mean_absolute_error: 1.4775 - mean_squared_error: 2.8724 - val_loss: 2.8916 - val_mean_absolute_error: 1.4781 - val_mean_squared_error: 2.8916
Epoch 3/3500
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 2.8779 - mean_absolute_error: 1.4766 - mean_squared_error: 2.8779 - val_loss: 2.8753 - val_mean_absolute_error: 1.4737 - val_mean_squared_error: 2.8753
Epoch 4/3500
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - loss: 2.8360 - mean_absolute_error: 1.4701 - mean_squared_error: 2.8360 - val_loss: 2.4153 - val_mean_absolute_error: 1.3616 - val_mean_squared_error: 2.4153
Epoch 5/3500
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 2.2206 - mean_absolute_error: 1.2891 - mean_squared_error: 2.2206 - val_loss: 1.6092 - 

In [9]:
def regression_results(y_true, y_pred):
    # Regression metrics
    ev = explained_variance_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred) 
    mse = mean_squared_error(y_true, y_pred) 
    r2 = r2_score(y_true, y_pred)

    print('explained_variance: ', round(ev,4))    
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('R²: ', round(r2,4))

In [10]:
# 7. Evaluamos el modelo

pred_tr = model.predict(X_train)
pred_te = model.predict(X_test)

#%% Model Performance Summary    
print("")
print('---------- Evaluation on Training Data ----------')
regression_results(y_train, pred_tr)
print("")

print('---------- Evaluation on Test Data ----------')
regression_results(y_test, pred_te)
print("")

#%% Results per output
mse_train = mean_squared_error(y_train, pred_tr)

r2_train = r2_score(y_train, pred_tr)

mse_test = mean_squared_error(y_test, pred_te)

r2_test = r2_score(y_test, pred_te)

col_names = ('MSE (train)', 'R2 (train)', 'MSE (test)', 'R2 (test)')

df = np.array([mse_train, r2_train, mse_test, r2_test])

print("")
print('---------- Evaluation per output ----------')
results = pd.DataFrame(data = df.reshape(1,-1), columns = col_names)
print(results)

438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step

---------- Evaluation on Training Data ----------
explained_variance:  0.8416
MAE:  0.4928
MSE:  0.4527
R²:  0.8416

---------- Evaluation on Test Data ----------
explained_variance:  0.8449
MAE:  0.4909
MSE:  0.4493
R²:  0.8449


---------- Evaluation per output ----------
   MSE (train)  R2 (train)  MSE (test)  R2 (test)
0      0.45271    0.841556    0.449328    0.84486


In [ ]:
# Prueba de pendiente-intercepto